In [3]:
pip install tensorflow opencv-python numpy matplotlib

In [1]:
import cv2
import numpy as np

def preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB if needed
    img = cv2.resize(img, target_size)  # Resize to specified dimensions
    img = img.astype(np.float32) / 255.0  # Normalize pixel values
    return img

In [2]:
import tensorflow as tf 
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, Dropout, Flatten

model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # Load pre-trained MobileNet
for layer in model.layers[:-5]:  # Freeze initial layers
    layer.trainable = False

x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)  # Add new layers
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)  # Output layer for 4 classes

model = tf.keras.Model(inputs=model.input, outputs=predictions)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths to the training and validation directories
train_dir = r'C:\Users\vidhya\Downloads\archive\PlantVillage\Train'
validation_dir = r'C:\Users\vidhya\Downloads\archive\PlantVillage\validation'

# Define image size and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Define data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators for training and validation
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

# Save the model
model.save('plant_disease_model.h5')

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('plant_disease_model.tflite', 'wb') as f:
    f.write(tflite_model)


Found 20638 images belonging to 4 classes.
Found 20638 images belonging to 4 classes.
Epoch 1/5


C:\Users\vidhya\.keras\New folder\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


645/645 ━━━━━━━━━━━━━━━━━━━━ 1378s 2s/step - accuracy: 0.9535 - loss: 0.1556 - val_accuracy: 0.9766 - val_loss: 0.0588
Epoch 2/5
645/645 ━━━━━━━━━━━━━━━━━━━━ 1008s 2s/step - accuracy: 0.9830 - loss: 0.0472 - val_accuracy: 0.9855 - val_loss: 0.0411
Epoch 3/5
645/645 ━━━━━━━━━━━━━━━━━━━━ 926s 1s/step - accuracy: 0.9850 - loss: 0.0414 - val_accuracy: 0.9801 - val_loss: 0.0587
Epoch 4/5
645/645 ━━━━━━━━━━━━━━━━━━━━ 731s 1s/step - accuracy: 0.9858 - loss: 0.0381 - val_accuracy: 0.9796 - val_loss: 0.0521
Epoch 5/5
645/645 ━━━━━━━━━━━━━━━━━━━━ 921s 1s/step - accuracy: 0.9883 - loss: 0.0318 - val_accuracy: 0.9862 - val_loss: 0.0381


INFO:tensorflow:Assets written to: C:\Users\vidhya\AppData\Local\Temp\tmpruj6dd_a\assets


INFO:tensorflow:Assets written to: C:\Users\vidhya\AppData\Local\Temp\tmpruj6dd_a\assets
